In [27]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import pandas as pd
df_raw = pd.read_parquet('data.parquet', engine='pyarrow')

In [ ]:
df_raw.head()

,write_time,chat_history,last_user_utterance,language,label
0,2024-08-05 14:45:22.123000 UTC,"[{""content"": ""Hello! Where can I find a good r...",Can you suggest a nice place for Italian food?,English,search_pois
1,2024-08-15 12:34:45.123000 UTC,"[{""content"": ""Hey there!"", ""additional_kwargs""...",Are there any good Italian restaurants around ...,English,search_pois
2,2024-08-01 10:15:45.123000 UTC,[],Can you suggest some local restaurants for din...,English,search_pois
3,2024-08-10 14:30:45.567000 UTC,"[{""content"": ""Hi there! What can I help you wi...",Do you know where I can find a good sci-fi boo...,English,search_pois
4,2024-07-30 15:15:00.123000 UTC,[],Can you recommend a good café nearby?,English,search_pois


In [ ]:
df_raw['language'].value_counts()

,count
language,
English,174
Japanese,30
Korean,25
Spanish,5
French,5
German,1


In [ ]:
df_raw['label'].value_counts()

,count
label,
search_music,50
control_navigation,50
no_action,50
search_pois,40
search_poi,30
drivers_manual,20


In [ ]:
df_raw.shape

(240, 5)

In [ ]:
import pandas as pd
import torch
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)




In [ ]:
json.loads(df_raw[1:2].chat_history.tolist()[0])

[{'content': 'Hey there!',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None,
  'example': False},
 {'content': 'Hello! What can I help you with today?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': []},
 {'content': "I'm looking for a good restaurant for dinner.",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None,
  'example': False},
 {'content': 'What type of cuisine do you prefer?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': []},
 {'content': 'I feel like having some Italian food.',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None,
  'example': False},
 {'content': 'Great choice! Let me find 

In [ ]:
json.loads(df_raw.chat_history.head(1).tolist()[0])

[{'content': 'Hello! Where can I find a good restaurant nearby?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None,
  'example': False},
 {'content': 'Hi there! What type of cuisine are you in the mood for?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': []},
 {'content': "I'm craving some Italian food.",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None,
  'example': False},
 {'content': 'There are a few great Italian restaurants around you, such as Trattoria Al Forno and Osteria Morini. Would you like more details?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': []},
 {'content': 'Yes, please provide the addresses.',
  'additional_kwargs': {},
  're

In [ ]:
for idx, row in df_raw[:5].iterrows():
  print(json.loads(row.chat_history))
  print(' [SEP] '.join(x['content'] for x in json.loads(row['chat_history'])) if row['chat_history'] else '')

[{'content': 'Hello! Where can I find a good restaurant nearby?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}, {'content': 'Hi there! What type of cuisine are you in the mood for?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': []}, {'content': "I'm craving some Italian food.", 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}, {'content': 'There are a few great Italian restaurants around you, such as Trattoria Al Forno and Osteria Morini. Would you like more details?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': []}, {'content': 'Yes, please provide the addresses.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None,

In [ ]:
# Optional: combine chat history and last user utterance
def combine_context(row):
    chat_history = ' [SEP] '.join(x['content'] for x in json.loads(row['chat_history']))
    return f"{chat_history} [SEP] {row['last_user_utterance']}"

In [ ]:
df_raw['combined_text'] = df_raw.apply(combine_context, axis=1)

# Encode labels
label2id = {label: idx for idx, label in enumerate(df_raw['label'].unique())}
id2label = {v: k for k, v in label2id.items()}
df_raw['label_id'] = df_raw['label'].map(label2id)


In [ ]:
# !python -m spacy download fr_core_news_sm
# !python -m spacy download es_core_news_sm
# !python -m spacy download ko_core_news_sm
# !python -m spacy download de_core_news_sm
# !python -m spacy download ja_core_news_sm

In [ ]:
import re
import spacy

# Load spaCy models for supported languages
spacy_models = {
    'English': spacy.load('en_core_web_sm'),
    'French': spacy.load('fr_core_news_sm'),
    'Spanish': spacy.load('es_core_news_sm'),
    'Korean': spacy.load('ko_core_news_sm'),
    'German': spacy.load('de_core_news_sm'),
    'Japanese': spacy.load('ja_core_news_sm')
}

In [ ]:


def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_named_entities(text, lang_name):
    nlp = spacy_models.get(lang_name)
    if not nlp:
        return text  # fallback: skip NER if no model
    doc = nlp(text)
    new_tokens = []
    for token in doc:
        if token.ent_type_:
            new_tokens.append(f"<{token.ent_type_}>")
        else:
            new_tokens.append(token.text)
    return " ".join(new_tokens)

def preprocess_multilingual(text, lang_name):
    text = remove_numbers(text)
    text = remove_named_entities(text, lang_name)
    return text


In [ ]:
#df['clean_text'] = df.apply(lambda row: preprocess_multilingual(row['text_str'], row['language']), axis=1)

In [ ]:
df_only_utterance_raw = df_raw.copy()
df_combined_clean = df_raw.copy()
df_only_utterance_clean = df_raw.copy()
df_raw['text_str'] = df_raw['combined_text'].astype(str)
df_only_utterance_raw['text_str'] = df_only_utterance_raw['last_user_utterance'].astype(str)
df_combined_clean['text_str'] = df_combined_clean.apply(lambda row: preprocess_multilingual(row['combined_text'], row['language']), axis=1)
df_only_utterance_clean['text_str'] = df_only_utterance_clean.apply(lambda row: preprocess_multilingual(row['last_user_utterance'], row['language']), axis=1)
df = pd.concat([df_raw, df_only_utterance_raw, df_combined_clean, df_only_utterance_clean])

In [ ]:
df.shape

(960, 9)

In [ ]:
# for idx, row in df[:5].iterrows():
#   print(row.clean_text)
#   print(synonym_replacement(row.clean_text))
#   print('\n')


In [ ]:

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text_str'], df['label_id'], test_size=0.2, stratify=df['label_id'], random_state=42
)

In [ ]:

# 2. Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

# 3. Dataset class
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, list(train_labels))
val_dataset = IntentDataset(val_encodings, list(val_labels))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

# 4. Model and training setup
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir='./results3',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    eval_strategy="epoch",
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

def compute_metrics(eval_pred):
    preds = eval_pred.predictions.argmax(axis=1)
    labels = eval_pred.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# 5. Train and evaluate
trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: williamdaburke (williamdaburke-burke-consulting-ltd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.774500,0.697274,0.744792,0.617171
2,0.211000,0.419033,0.901042,0.895565
3,0.256700,0.299311,0.901042,0.890251
4,0.110500,0.350736,0.911458,0.913965
5,0.074900,0.142861,0.963542,0.963790
6,0.001300,0.161513,0.968750,0.966328
7,0.001000,0.157598,0.963542,0.961858
8,0.000800,0.163137,0.958333,0.957302


KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate()
print("Final evaluation:", metrics)

NameError: name 'trainer' is not defined

In [ ]:
!rm -r ./results/


In [ ]:

# 6. Save model and tokenizer
model.save_pretrained("./intent_classifier_distilbert")
tokenizer.save_pretrained("./intent_classifier_distilbert")


In [ ]:
trainer.save_model("./intent_classifier_distilbert_model")

In [ ]:
trainer.train(resume_from_checkpoint="./intent_classifier_distilbert_model")

In [ ]:
# Find the latest checkpoint in the output directory to resume from
import glob
import os

# Get the output directory from training_args
output_dir = training_args.output_dir

# Find all directories within the output_dir that look like checkpoints
checkpoints = sorted(glob.glob(os.path.join(output_dir, "checkpoint-*")), key=os.path.getmtime)

# Check if any checkpoints were found
if checkpoints:
    # Get the path to the latest checkpoint directory
    latest_checkpoint_path = checkpoints[-1]
    print(f"Resuming training from the latest checkpoint: {latest_checkpoint_path}")
    # Resume training from the latest checkpoint
    trainer.train(resume_from_checkpoint=latest_checkpoint_path)
else:
    print("No checkpoints found in the output directory to resume from.")
    # Optionally, you might want to handle this case, e.g., train from scratch again
    # trainer.train()